In [24]:
import os
import librosa
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.io import wavfile
import noisereduce as nr
from scipy.io.wavfile import read
from sklearn.neighbors import KNeighborsClassifier


In [25]:
absolute_path = os.path.dirname(
    "E:/SOFT/General_Subjects/artificial_intelligence/colab-tts-Bahnar/bahnar_tts/")
relative_path = "pattern"

fullpath = os.path.join(absolute_path, relative_path)


In [26]:
for count, filename in enumerate(os.listdir(fullpath)):
    print(filename)


.DS_Store
AnhSon
ChiBi


In [27]:
def calc_distances(sound_file):
    #The minimun value for the sound to be recognized as a knock
    min_val = 5000
    
    fs, data = read(sound_file)
    data_size = len(data)
    
    #The number of indexes on 0.15 seconds
    focus_size = int(0.15 * fs)
    
    focuses = []
    distances = []
    idx = 0
    
    while idx < len(data):
        if data[idx] > min_val:
            mean_idx = idx + focus_size // 2
            focuses.append(float(mean_idx) / data_size)
            if len(focuses) > 1:
                last_focus = focuses[-2]
                actual_focus = focuses[-1]
                distances.append(actual_focus - last_focus)
            idx += focus_size
        else:
            idx += 1
    return distances

In [28]:
def accept_test(pattern, test, min_error):
    if len(pattern) > len(test):
        return False
    for i, dt in enumerate(pattern):
        if not dt - test[i] < min_error:
            return False
    return True

### plot wav files

In [ ]:
x = 0
y = 0
fig, axes = plt.subplots(4, 4, figsize=(20, 10))

for count, filename in enumerate(os.listdir(fullpath)):
    if filename == "AnhSon" or filename == "ChiBi":
        subpath = os.path.join(fullpath, filename)
        for c, f in enumerate(os.listdir(subpath)):
            if f == "conv-wav":
                subsubpath = os.path.join(subpath, f)
                for c1, f1 in enumerate(os.listdir(subsubpath)):
                    audiopath = os.path.join(subsubpath, f1)
                    print(audiopath)
                    fs, data = read(audiopath)
                    input_data = read(audiopath)
                    audio = input_data[1]
                    axes[x, y].plot(audio)

                    # load data
                    r, d = wavfile.read(f"{audiopath}")
                    # perform noise reduction
                    reduced_noise = nr.reduce_noise(y=d, sr=r)
                    wavfile.write(f"{subpath}/reduce-noise/test{c1}.wav", r, reduced_noise)

                    # do a comparison of the reduced noise files
                    adjusted_filepath = os.path.join(subpath, "reduce-noise")
                    adjusted_filepath = os.path.join(adjusted_filepath, f"test{c1}.wav")
                    fs, data = read(adjusted_filepath)
                    input_data = read(adjusted_filepath)
                    audio = input_data[1]
                    axes[x, y].plot(audio)

                    y = y + 1
                    if y == 4:
                        y = 0
                        x = x + 1



### detect dataset filepath

In [ ]:
dataset = os.path.join(absolute_path, 'dataset')
for count, filename in enumerate(os.listdir(dataset)):
    print(filename)

### Process AnhSon's audio files

In [ ]:
for count, filename in enumerate(os.listdir(fullpath)):
    if filename == "AnhSon":
        subpath = os.path.join(fullpath, filename)
        for c, f in enumerate(os.listdir(subpath)):
            if f == "reduce-noise":
                subsubpath = os.path.join(subpath,f)
                for c1, f1 in enumerate(os.listdir(subsubpath)):
                    if f1.endswith('wav'):
                        audiopath = os.path.join(subsubpath, f1)
                        print(audiopath)
                        fs, data = read(audiopath)
                        # we will use the size of the array
                        # to determine the duration of the sound
                        data_size = len(data)

                        # build k-nearest neighbors
                        # ...

                        pattern = calc_distances(f1)
                        test = calc_distances( ) #filename to be processed

                        min_error = 0.1
                        
                        print(accept_test(pattern, test, min_error))

### plot wav files

### noise reducion